In [ ]:
#import libraries
import pandas as pd
import numpy as np
import json
import requests
import networkx
import scipy

In [ ]:
#create empty dataframe, enter api key and show name as variables
#enter show search url as variable
#search for show id 
df_master = pd.DataFrame()
api_key = "k_793904nk/" 
show = "Wynonna Earp" 
show_search_url = "https://imdb-api.com/en/API/SearchTitle/" + api_key + show
s = requests.get(show_search_url)
s.json()
#flatten s1
s1 = json.loads(s.text)
df1 = pd.json_normalize(s1, ['results'])
#print (df)
show_id = df1.loc[df1['title'] == show,'id'].item()
print (show_id)

In [ ]:
#enter cast search url as variable
#search for cast members and ids
cast_search_url = "https://imdb-api.com/en/API/FullCast/" + api_key + show_id
c = requests.get(cast_search_url)
c.json()
#flatten c1 and extract actors
c1 = json.loads(c.text)
df2 = pd.json_normalize(c1, ['actors'])
df2.drop(['image', 'asCharacter'], axis=1, inplace=True)
df2 = df2.drop_duplicates()
#print (df2)
df4 = df_master.append(df2, ignore_index = True)
#extract writers and directors; append it to the actors list; drop duplicates
c.json()
df3 = pd.json_normalize(c1,  record_path =['directors', 'items'])
df3.drop('description', axis=1, inplace=True)
df3 = df3.drop_duplicates()
#print (df3)
df5 = df4.append(df3, ignore_index = True)
df5 = df5.drop_duplicates()
#print (df6)
df6 = pd.json_normalize(c1,  record_path =['writers', 'items'])
df6.drop('description', axis=1, inplace=True)
df6 = df6.drop_duplicates()
#print (df6)
df7 = df5.append(df6, ignore_index = True)
df7 = df7.drop_duplicates()
print (df7)


In [ ]:
#extract cast ids and names into lists for iterative queries
cast_ids = df7['id'].to_list()
cast_names = df7['name'].to_list()
#print (cast_names)
#print (cast_ids)
#loop test 
edges = []
for cast_id in cast_ids:
    print (cast_id)
    name_search_url = "https://imdb-api.com/en/API/Name/" + api_key + cast_id
    #print(name_search_url)
    n = requests.get(name_search_url)
    n.json()
    n1 = json.loads(n.text)
    #print (n1)
    dfn = pd.json_normalize(n1, 'castMovies',meta=['name'])
    dfn.drop(['role', 'year', 'description'], axis=1, inplace=True)
    dfn['cast_id'] = cast_id
    
    edges.append(dfn)
    #print (edges)
    #if cast_id == 'nm3532853':
        #break 
edges_final = pd.concat(edges)
print (edges_final)


In [12]:
import networkx as nx
G = nx.from_pandas_edgelist(edges_final, 
                            source='name',
                            target='title')
#nx.draw_spring(G)
#nx.draw(G, pos=nx.spring_layout(G))

In [13]:
#import pyvis
from pyvis.network import Network
#create vis network
net = Network(notebook=True)
#load the networkx graph
net.from_nx(G)
#show
net.show("cast.html")

ModuleNotFoundError: No module named 'pyvis'

In [ ]:
from pathlib import Path
filepath = Path('edges.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
edges_final.to_csv(filepath)


In [ ]:
import dash
import visdcc
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State

#create app
app = dash.Dash()

#load data edges_final
df9 = pd.read_csv("edges.csv")
node_list = list(set(df9['name'].unique().tolist() + 
                     df9['title'].unique().tolist())
                )
nodes = [{'id': node_name, 'label': node_name, 'shape': 'dot', 'size': 7}
        for i, node_name in enumerate(node_list)]
#create edges from df
edges2 = []
for row in df9.to_dict(orient='records'):
    source, target = row['name'], row['title']
    edges2.append({
        'id': source + "_" + target, 
        'from': source,
        'to': target, 
        'width': 2, 
    })
    
#define layout
app.layout = html.Div([
    visdcc.Network(id = 'net',
                   data = {'nodes': nodes, 'edges': edges2},
                   options = dict(height= '600px', width= '100%')),
    dcc.RadioItems(id = 'color',
                   options=[{'label': 'Red', 'value': '#ff0000'},
                             {'label': 'Green', 'value': '#00ff00'},
                             {'label': 'Blue', 'value': '#0000ff'}],
                   value='Red')
])

#define callback
@app.callback(
    Output('net', 'options'),
    [Input('color', 'value')])
def myfun(x):
    return {'nodes':{'color': x}}

#define main calling
if __name__ == '__main__':
    app.run_server(debug=True)
